# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
lat_lng_list = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        lat_lng_list.append(lat_lng)

# Print the city count to confirm sufficient count
len(cities)

651

In [3]:
print(cities)

['narsaq', 'rikitea', 'faanui', 'taolanaro', 'coihueco', 'mataura', 'klaksvik', 'castro', 'khatanga', 'yelovo', 'manaure', 'chuy', 'atuona', 'bredasdorp', 'bluff', 'bambous virieux', 'saint anthony', 'porterville', 'luebo', 'new norfolk', 'albany', 'enger', 'padang', 'puerto baquerizo moreno', 'sunrise manor', 'hobart', 'georgetown', 'zyryanka', 'emerald', 'vao', 'ponta do sol', 'dikson', 'ribeira grande', 'bengkulu', 'xai-xai', 'tasiilaq', 'saskylakh', 'lavrentiya', 'chumikan', 'monte carmelo', 'yar-sale', 'vardo', 'jamestown', 'cape town', 'attawapiskat', 'airai', 'chokurdakh', 'palabuhanratu', 'kavieng', 'east london', 'berlevag', 'cherskiy', 'kamenka', 'port alfred', 'shahr-e babak', 'fortuna', 'san rafael', 'kloulklubed', 'lamont', 'illoqqortoormiut', 'kirakira', 'ahipara', 'puerto ayora', 'grindavik', 'provideniya', 'robe', 'svetlaya', 'barrow', 'camacha', 'half moon bay', 'araouane', 'te anau', 'caraballeda', 'gongzhuling', 'chebsara', 'toowoomba', 'kisangani', 'merauke', 'tagan

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
#Create dataframe with list of cities
df = pd.DataFrame(cities)
df = df.rename(columns={0: 'city'})

#Add lat and lngs to dataframe, create separate columns for lats and longs
df['lat_lngs'] = lat_lng_list
df['lat'] = df.lat_lngs.map(lambda x: str(x[0]))
df['long'] = df.lat_lngs.map(lambda x: str(x[1]))

df.head()

,city,lat_lngs,lat,long
0,narsaq,"(89.14357451521826, -58.83449868033965)",89.14357451521826,-58.83449868033965
1,rikitea,"(-38.33721922886667, -113.60634601783528)",-38.33721922886667,-113.60634601783528
2,faanui,"(-5.303792295267542, -159.21100595147885)",-5.303792295267542,-159.21100595147885
3,taolanaro,"(-77.12754864368091, 61.74802231422211)",-77.12754864368091,61.74802231422211
4,coihueco,"(-37.425134550438045, -70.03887264713072)",-37.425134550438045,-70.03887264713072


In [5]:
#Create new columns for data we'll be collecting from the API
df['temp'] = ""
df['max_temp'] = ""
df['humidity'] = ""
df['wind_speed'] = ""
df['clouds'] = ""

#Iterate over each row
for index, row in df.iterrows():
    city = row['city']
    print(f"Processing Record {index + 1} | {city}")
    city = city.replace(" ", "&")
    url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=" + city + "&APPID=" + weather_api_key
    print(url)
    weather = requests.get(url).json()
    try:
        df.loc[index, 'temp'] = weather['main']['temp']
        df.loc[index, 'max_temp'] = weather['main']['temp_max']
        df.loc[index, 'humidity'] = weather['main']['humidity']
        df.loc[index, 'wind_speed'] = weather['wind']['speed']
        df.loc[index, 'clouds'] = weather['clouds']['all']
    except:
        df.loc[index, 'temp'] = 'city not found'
        df.loc[index, 'humidity'] = 'city not found'
        df.loc[index, 'wind_speed'] = 'city not found'
        df.loc[index, 'clouds'] = 'city not found'
    time.sleep(.50)
    
print("----------------------")
print("Data Retrieval Complete")
print("-----------------------")

Processing Record 1 | narsaq
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=narsaq&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 2 | rikitea
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=rikitea&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 3 | faanui
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=faanui&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 4 | taolanaro
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=taolanaro&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 5 | coihueco
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=coihueco&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 6 | mataura
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=mataura&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 7 | klaksvik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=klaksvik&APPID=cf45ad5d60076af179f9f3166bb272c9
Processin

Processing Record 58 | kloulklubed
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=kloulklubed&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 59 | lamont
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=lamont&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 60 | illoqqortoormiut
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=illoqqortoormiut&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 61 | kirakira
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=kirakira&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 62 | ahipara
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=ahipara&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 63 | puerto ayora
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=puerto&ayora&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 64 | grindavik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=grindavik&APPID

Processing Record 115 | utiroa
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=utiroa&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 116 | torbay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=torbay&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 117 | manturovo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=manturovo&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 118 | esperance
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=esperance&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 119 | manggar
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=manggar&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 120 | petropavlovsk-kamchatskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=petropavlovsk-kamchatskiy&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 121 | vaini
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=vaini&AP

Processing Record 171 | bethel
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=bethel&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 172 | hovd
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=hovd&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 173 | saint george
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=saint&george&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 174 | katsuura
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=katsuura&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 175 | baykit
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=baykit&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 176 | talnakh
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=talnakh&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 177 | carmen
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=carmen&APPID=cf45ad5d60076af179f9f3166bb272c9

Processing Record 228 | filingue
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=filingue&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 229 | menongue
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=menongue&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 230 | mukhen
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=mukhen&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 231 | maldonado
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=maldonado&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 232 | waipawa
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=waipawa&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 233 | college
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=college&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 234 | timizart
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=timizart&APPID=cf45ad5d60076af179f9f316

...

Processing Record 398 | ciudad bolivar
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=ciudad&bolivar&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 399 | vung tau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=vung&tau&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 400 | falam
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=falam&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 401 | jalu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=jalu&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 402 | qui nhon
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=qui&nhon&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 403 | dabakala
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=dabakala&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 404 | saint-pierre
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=saint-pierre&APPID=cf45ad5d6007

Processing Record 455 | west odessa
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=west&odessa&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 456 | broome
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=broome&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 457 | amazar
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=amazar&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 458 | sibolga
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=sibolga&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 459 | kushmurun
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=kushmurun&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 460 | nizhneyansk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=nizhneyansk&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 461 | cape coast
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=cape&coast&APPID=cf45ad5d60

Processing Record 512 | villarrica
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=villarrica&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 513 | honningsvag
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=honningsvag&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 514 | mezen
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=mezen&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 515 | porto santo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=porto&santo&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 516 | sherwood
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=sherwood&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 517 | kwidzyn
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=kwidzyn&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 518 | gazli
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=gazli&APPID=cf45ad5d60076af1

Processing Record 569 | babanusah
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=babanusah&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 570 | coventry
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=coventry&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 571 | tabinay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=tabinay&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 572 | preobrazheniye
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=preobrazheniye&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 573 | aberystwyth
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=aberystwyth&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 574 | leonidion
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=leonidion&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 575 | thessalon
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=thessalon&AP

Processing Record 625 | sochi
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=sochi&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 626 | ondorhaan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=ondorhaan&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 627 | astoria
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=astoria&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 628 | otjiwarongo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=otjiwarongo&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 629 | astana
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=astana&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 630 | barahona
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=barahona&APPID=cf45ad5d60076af179f9f3166bb272c9
Processing Record 631 | christchurch
http://api.openweathermap.org/data/2.5/weather?units=Imperial&q=christchurch&APPID=cf45ad5d60076a

In [6]:
#Remove any cities that could not be found from the dataframe
df = df[df.temp != 'city not found']

#Check that there are still 500+ records
len(df)

586

In [7]:
#Convert lat from string to float object
df.lat = df.lat.astype(float)
df.head()

,city,lat_lngs,lat,long,temp,max_temp,humidity,wind_speed,clouds
0,narsaq,"(89.14357451521826, -58.83449868033965)",89.143575,-58.83449868033965,46.4,46.4,93,4.7,100
1,rikitea,"(-38.33721922886667, -113.60634601783528)",-38.337219,-113.60634601783528,72.5,72.5,82,14.09,95
2,faanui,"(-5.303792295267542, -159.21100595147885)",-5.303792,-159.21100595147885,79,79,80,12.97,88
4,coihueco,"(-37.425134550438045, -70.03887264713072)",-37.425135,-70.03887264713072,39.2,39.2,80,8.05,0
5,mataura,"(-51.71691900586968, -153.47207457999704)",-51.716919,-153.47207457999704,46.99,46.99,81,1.01,100


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [8]:
df.to_csv(output_data_file, encoding="utf-8", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
#Pull in today's date for graphs
date = datetime.date.today()
date = time.strftime("%m/%d/%Y")

sns.set()
plt.figure(figsize=(10,8))
plt.scatter(df['lat'], df['temp'])
plt.title(f"City Latitude vs. Temperature {date}", fontsize="18")
plt.xlabel("Latitude", fontsize="14")
plt.ylabel("Temperature (F)", fontsize="14")
plt.ylim(0, 120)

plt.savefig("Temperature.png")

plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(df['lat'], df['humidity'])                              
plt.title(f"City Latitude vs. Humidity {date}", fontsize="18")
plt.xlabel("Latitude", fontsize="14")
plt.ylabel("Humidity (%)", fontsize="14")

plt.ylim(0,120)

plt.savefig("Images/Humidity.png")

plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(df['lat'], df['clouds'])                              
plt.title(f"City Latitude vs.Cloudiness {date}", fontsize="18")
plt.xlabel("Latitude", fontsize="14")
plt.ylabel("Wind Speed (mph)", fontsize="14")
plt.ylim(-20, 120)

plt.savefig("Cloudiness.png")

plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(df['lat'], df['wind_speed'])                              
plt.title(f"City Latitude vs. Wind Speed {date}", fontsize="18")
plt.xlabel("Latitude", fontsize="14")
plt.ylabel("Wind Speed (mph)", fontsize="14")

plt.ylim(-5,45)

plt.savefig("Wind_Speed.png")

plt.show()